In [1]:
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
app =  dash.Dash()
server =  app.server
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

In [2]:
df = pd.read_csv('amazon.csv',encoding='latin1')

#MONTHS TO ENGLISH
span_to_eng = {
    'Janeiro':'January',
    'Fevereiro': 'February',
    'Março':'March',
    'Abril':'April',
    'Maio':'May',
    'Junho':'June',
    'Julho':'July',
    'Agosto':'August',
    'Setembro':'September',
    'Outubro': 'October',
    'Novembro': 'November',
    'Dezembro':'December'    
}
df['month'] = df['month'].map(span_to_eng)
available_indicators = df['month'].unique()
df.head()

,year,state,month,number,date
0,1998,Acre,January,0.0,1998-01-01
1,1999,Acre,January,0.0,1999-01-01
2,2000,Acre,January,0.0,2000-01-01
3,2001,Acre,January,0.0,2001-01-01
4,2002,Acre,January,0.0,2002-01-01


In [3]:
df.isnull().sum()

year      0
state     0
month     0
number    0
date      0
dtype: int64

In [4]:
df_max = pd.DataFrame(df.groupby(['year','state','month']).number.sum())


In [5]:
df_sum_state = pd.DataFrame(df_max.groupby(['state']).number.sum())
df_sum_state = df_sum_state[df_sum_state.number == df_sum_state.number.max()]
print('The state with the most fires is: ')
df_sum_state.reset_index(drop=False)

The state with the most fires is: 


,state,number
0,Mato Grosso,96246.028


In [6]:
df_sum_year = pd.DataFrame(df_max.groupby(['year']).number.sum())
df_sum_yea = df_sum_year[df_sum_year.number == df_sum_year.number.max()]
print('The year with most fires is:')
df_sum_yea

The year with most fires is:


,number
year,
2003,42760.674


In [7]:
df['key'] = df.month.map(str)+'-'+df.year.apply(str)
df_sum_yr_month = pd.DataFrame(df.groupby(['key']).number.sum())
df_sum_yr_mont = df_sum_yr_month[df_sum_yr_month.number == df_sum_yr_month.number.max()]
print('The year and month with most fires is:')
df_sum_yr_mont.reset_index(drop=False)

The year and month with most fires is:


,key,number
0,August-2006,7338.127


In [8]:
df['key'] = df.month.map(str)+'-'+df.year.apply(str)
df['country']= 'Brazil'
hover_text = []

#define hover text
for index, row in df.iterrows():
    hover_text.append(('Country: {country}<br>'+
                      'State: {state}<br>'+
                      'Fires: {number}<br>'+
                      'Year: {year}').format(country=row['country'],
                                            state=row['state'],
                                            number=row['number'],
                                            year=row['year']))
df['text'] = hover_text


In [ ]:
df_acre = df[df.state =='Acre']
df_alagoas = df[df.state =='Alagoas']
df_amapa = df[df.state =='Amapa']
df_amazonas = df[df.state =='Amazonas']
df_bahia = df[df.state =='Bahia']
df_ceara = df[df.state =='Ceara']
df_distrito = df[df.state =='Distrito Federal']
df_esperito = df[df.state =='Esperito Santo']
df_goias = df[df.state =='Goias']
df_maranhao = df[df.state =='Maranhao']
df_mato = df[df.state =='Mato Grosso']
df_minas = df[df.state =='Minas Gerais']
df_para = df[df.state =='Pará']
df_paraiba= df[df.state =='Paraiba']
df_perna = df[df.state =='Pernambuco']
df_goias = df[df.state =='Piau']
df_rio = df[df.state =='Rio']
df_rondo = df[df.state =='Rondonia']
df_roraima = df[df.state =='Roraima']
df_santa = df[df.state =='Santa Catarina']
df_sao = df[df.state =='Sao Paulo']
df_sergipe = df[df.state =='Sergipe']
df_tocantins = df[df.state =='Tocantins']

In [ ]:

app.layout = html.Div(style={'backgroundColor': ['#b50000', '#e63535', '#fa8989'],'size': 9,'line': {'width': 1, 'color': 'orange'}}, children=[
    html.H1(
        children='Brazil Forest Fires',
        style={
            'textAlign': 'center',
            'color': 'green',
            'backgroundColor': 'yellow'
        }
    ),
    html.Div(
            [
            html.Div(
                [
                    html.H1('Fires per state', style={'textAlign': 'center','color': 'green','backgroundColor': 'yellow'}),
                    dcc.RadioItems(
                            id = 'stats',
                            options=[
                                {'label': 'Fire per state', 'value': 'state'},
                                {'label': 'Fire per year', 'value': 'year'},
                                {'label': 'Fire per month', 'value': 'month'}
                            ],
                            value=['state', 'year','month'],
                            labelStyle={'display': 'inline-block','color': 'green'}
                    ),
                ],
                className='six columns',
                style={'margin-top': '10'}
            ),
        ], className="row"
    ),
    dcc.Graph(
        id='Graph1',
        figure={
            'data': [
                {'x': df.state, 'y': df.number, 'type': 'bar', 'name': 'state','marker': {'color': ['#b50000', '#e63535', '#fa8989'],'size': 9,'line': {'width': 1, 'color': 'orange'}}},
                
            ],
            'layout': {
                'plot_bgcolor': colors['background'],
                'paper_bgcolor': colors['background'],
                'font': {
                'color': 'green'
                }
            }
        }
    ),
      html.Div(style={'backgroundColor': 'yellow'}, children=[
    html.H1(
        children='Fires per year',
        style={
            'textAlign': 'center',
            'color': 'green'
        }
    )]),
    dcc.Graph(
        id='Graph2',
        figure={
            'data': [
                {'x': df.year, 'y': df.number, 'type': 'bar', 'name': 'year','marker': {'color': ['#b50000', '#e63535', '#fa8989'],'size': 9,'line': {'width': 1, 'color': 'orange'}}},
                
            ],
            'layout': {
                'plot_bgcolor': colors['background'],
                'paper_bgcolor': colors['background'],
                'font': {
                'color': 'green'
                }
            }
        }
    ),
      html.Div(style={'backgroundColor': 'yellow'}, children=[
    html.H1(
        children='Fires per Year and Month',
        style={
            'textAlign': 'center',
            'color': 'green'
        }
    )]),
    dcc.Graph(
        id='Graph3',
        figure={
            'data': [
                {'x': df.key, 'y': df.number, 'type': 'bar', 'name': 'month','marker': {'color': ['#b50000', '#e63535', '#fa8989'],'size': 9,'line': {'width': 1, 'color': 'orange'}}},
                
            ],
            'layout': {
                'plot_bgcolor': colors['background'],
                'paper_bgcolor': colors['background'],
                'font': {
                'color': 'green'
                }
            }
        }
    ), 
])
if __name__ == '__main__':
    app.run_server(port=4050)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Mar/2020 18:41:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 18:41:58] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 18:41:58] "GET /_favicon.ico?v=1.8.0 HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 18:41:58] "GET /_dash-layout HTTP/1.1" 200 -
